<a href="https://colab.research.google.com/github/grant88/education/blob/main/pandas_advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
from io import BytesIO

**Задание 1**

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

* если источник traffic_source равен yandex или google, то в source_type ставится organic

* для источников paid и email из России - ставим ad

* для источников paid и email не из России - ставим other

все остальные варианты берем из traffic_source без изменений

In [ ]:
filename = "visit_log.csv"
try:
    logs = pd.read_csv(f"content/{filename}", sep=";")
except:
    from google.colab import files
    print(f"Загрузите файл с логами просмотров {filename}\n")
    data = BytesIO(files.upload()[filename])
    logs = pd.read_csv(data, sep=";")

In [22]:
logs['source_type'] = logs.loc[:, 'traffic_source']

logs.loc[logs['traffic_source'].isin(['yandex', 'google']), 'source_type'] = 'organic'

logs.loc[(logs['traffic_source'].isin(['paid', 'email'])) & (logs['region'] == 'Russia'), 'source_type'] = 'ad'

logs.loc[(logs['traffic_source'].isin(['paid', 'email'])) & (logs['region'] != 'Russia'), 'source_type'] = 'other'

Проверка:

In [23]:
logs.source_type.value_counts()

direct     7013
organic    5407
ad         3790
other      2728
Name: source_type, dtype: int64

**Задание 2**

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

1. Прочитайте содержимое файла с датафрейм
2. Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [ ]:
filename = "URLs.txt"
try:
    urls = pd.read_csv(f"content/{filename}")
except:
    from google.colab import files
    print(f"Загрузите файл с страницами новостного сайта {filename}\n")
    data = BytesIO(files.upload()[filename])
    urls = pd.read_csv(data)

Отфильтрованные строки:

In [28]:
urls[urls.url.str.contains(pat='/\d{8}-', regex=True)]

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...
...,...
88,/cis/35984145-kreml-prokommentiroval-soobschen...
89,/video/36071019-olimpiyskie-obekty-rio-prevrat...
90,/science/36151301-nazvano-posledstvie-zloupotr...
91,/incidents/36027330-vospitatelnitsu-zatravili-...


**Задание 3**

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. 

Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [ ]:
filename = "ratings.csv"
try:
    ratings = pd.read_csv(f"content/{filename}")
except:
    from google.colab import files
    print(f"Загрузите файл с рейтингами фильмов {filename}\n")
    data = BytesIO(files.upload()[filename])
    ratings = pd.read_csv(data)

In [30]:
args = {
    'total_count': ('timestamp', 'count'),
    'difference': ('timestamp', lambda x: x.max() - x.min())
}

In [31]:
avg_time = ratings.groupby('userId').agg(**args).query('total_count > 100')['difference'].mean()

Среднее время жизни пользователя, дней:

In [32]:
round(avg_time / (24*60*60), 0)

464.0

**Задание 4**

Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

Необходимо сформировать две таблицы:

1. таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
2. аналогичную таблицу по типам выручки с указанием адреса клиента

In [33]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

In [34]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

In [35]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

In [36]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118, 9999],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4', 'dummy_address']
    }
)

In [37]:
revenue = pd.concat([rzd,auto, air]).groupby('client_id').sum().reset_index()

**Таблица 1.** Делаем merge, чтобы не потерять клиентов из client_base, по которым не было записи о какой-либо выручке

In [38]:
table_1 = client_base[['client_id']].merge(revenue, on='client_id', how='left').fillna(0)
table_1

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0
8,9999,0.0,0.0,0.0


**Таблица 2.** Заполняем нулями числовые столбцы, если были клиенты без выручки

In [39]:
table_2 = client_base.merge(revenue, on='client_id', how='left').fillna(0)
table_2

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,0.0,0.0
1,112,Энтузиастов 8а,2810.0,0.0,0.0
2,113,Левобережная 1а,10283.0,57483.0,0.0
3,114,Мира 14,5774.0,83.0,0.0
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,0.0,4834.0,4.0
6,117,Панфиловская 33,0.0,98.0,13.0
7,118,Мастеркова 4,0.0,0.0,173.0
8,9999,dummy_address,0.0,0.0,0.0
